In [ ]:
%load_ext autoreload

import matplotlib.pyplot as plt
import pickle
import sys
from astropy.table import join 
import numpy as np

import clmm
from clmm import Cosmology
import pyccl as ccl

sys.path.append('../modules/')
import CL_WL_DATAOPS_make_profile as prf


In [ ]:
cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
cosmo_ccl  = ccl.Cosmology(Omega_c=0.265-0.0448, Omega_b=0.0448, h=0.71, A_s=2.1e-9, n_s=0.96, Neff=0, Omega_g=0)

In [ ]:
from scipy.integrate import quad
def av_sigma_2(zl):
    def sigma_c_2(zs):
        return cosmo.eval_sigma_crit(zl, zs)**(2)
    def Chang(z): #Chang distribution of source redshift
        a = 1.24
        b = 1.01
        z_0 = 0.51
        return np.exp(-(z/z_0)**b)*z**a
    norm = quad(Chang,0,np.inf)[0]
    print(norm)
    def __INTEGRAND__(z): return Chang(z)*sigma_c_2(z)
    return quad(__INTEGRAND__,zl + 0.1,100)[0]/norm

In [ ]:
av_sigma_2(1)

In [ ]:
filename = '../data/individual_profile_cosmoDC2_gal_cosmoDC2_halo_epsilon_shape_with_number_density.pkl'
profile_nsf = pickle.load(open(filename, 'rb'))
filename = '../data/individual_profile_cosmoDC2_gal_cosmoDC2_halo_shear_shape.pkl'
profile_f = pickle.load(open(filename, 'rb'))
filename = '../data/lens_catalog_SkySim5000.pkl'
dat_SkySim = pickle.load(open(filename, 'rb'))

# profile_nsf = load('/pbs/throng/lsst/users/cpayerne/CLMassDC2/data/individual_profile_cosmoDC2_gal_cosmoDC2_halo_epsilon_shape_with_number_density.pkl')
# profile_f = load('/pbs/throng/lsst/users/cpayerne/CLMassDC2/data/individual_profile_cosmoDC2_gal_cosmoDC2_halo_shear_shape.pkl')
# dat_SkySim = load('/pbs/throng/lsst/users/cpayerne/CLMassDC2/data/lens_catalog_SkySim5000.pkl')

In [ ]:
profile_f.colnames

In [ ]:
profile_nsf['halo_id'] = profile_nsf['cluster_id']
profile_f['halo_id'] = profile_f['cluster_id']
profile_shapenoise = join(profile_nsf,dat_SkySim,keys='halo_id')
profile_noshapenoise = join(profile_f,dat_SkySim,keys='halo_id')

In [ ]:
m_SORT = np.sort(np.array(profile_f['M200c']), axis = 0)
m_list = np.array_split(m_SORT,5)
m_corner = []
for m_ in m_list[1:]:
    m_corner.append(m_[0][0])
m_corner.append(np.inf)

In [ ]:
def binning(corner): return [[corner[i],corner[i+1]] for i in range(len(corner)-1)]
def binning(corner): return [[corner[i],corner[i+1]] for i in range(len(corner)-1)]
z_corner = np.linspace(0.2, 1, 9)
Z_bin = binning(z_corner)
Mass_bin = binning(m_corner)

In [ ]:
z_corner

In [ ]:
n_boot = 300
n_jack = 150

In [ ]:
profile_stack_shapenoise = prf.stacked_profile(profile = profile_shapenoise,
                    r_in = 'radius_av',
                    gt_in = 'gt_av', gx_in = 'gx_av',
                    r_out = 'radius',
                    gt_out = 'gt', gx_out = 'gx',
                    weight = 'norm_sum',
                    z_name = 'cluster_z', obs_name = 'M200c',
                    Z_bin = Z_bin, Obs_bin = Mass_bin)
profile_stack_noshapenoise = prf.stacked_profile(profile = profile_noshapenoise,
                    r_in = 'radius_av',
                    gt_in = 'gt_av', gx_in = 'gx_av',
                    r_out = 'radius',
                    gt_out = 'gt', gx_out = 'gx',
                    weight = 'norm_sum',
                    z_name = 'cluster_z', obs_name = 'M200c',
                    Z_bin = Z_bin, Obs_bin = Mass_bin)

In [ ]:
covariance_stack_bootstrap_shapenoise = prf.bootstrap_covariance(profile = profile_shapenoise,
                    r_in = 'radius_av',
                    gt_in = 'gt_av', gx_in = 'gx_av',
                    r_out = 'radius',
                    gt_out = 'gt', gx_out = 'gx',
                    weight = 'norm_sum',
                    n_boot = n_boot,
                    z_name = 'cluster_z', obs_name = 'M200c',
                    Z_bin = Z_bin, Obs_bin = Mass_bin)

covariance_stack_bootstrap_noshapenoise = prf.bootstrap_covariance(profile = profile_noshapenoise,
                    r_in = 'radius_av',
                    gt_in = 'gt_av', gx_in = 'gx_av',
                    r_out = 'radius',
                    gt_out = 'gt', gx_out = 'gx',
                    weight = 'norm_sum',
                    n_boot = n_boot,
                    z_name = 'cluster_z', obs_name = 'M200c',
                    Z_bin = Z_bin, Obs_bin = Mass_bin)

print('ok boot')

covariance_stack_sample_shapenoise = prf.sample_covariance(profile = profile_shapenoise,
                    r_in = 'radius_av',
                    gt_in = 'gt_av', gx_in = 'gx_av',
                   r_out = 'radius',
                    gt_out = 'gt', gx_out = 'gx',
                    weight = 'norm_sum',
                    z_name = 'cluster_z', obs_name = 'M200c',
                    Z_bin = Z_bin, Obs_bin = Mass_bin)

covariance_stack_sample_noshapenoise = prf.sample_covariance(profile = profile_noshapenoise,
                    r_in = 'radius_av',
                    gt_in = 'gt_av', gx_in = 'gx_av',
                    r_out = 'radius',
                    gt_out = 'gt', gx_out = 'gx',
                    weight = 'norm_sum',
                    #n_boot = n_boot,                              
                    z_name = 'cluster_z', obs_name = 'M200c',
                    Z_bin = Z_bin, Obs_bin = Mass_bin)

print('ok sample')

covariance_stack_jacknife_shapenoise = prf.jacknife_covariance(profile = profile_shapenoise,
                    r_in = 'radius_av',
                    gt_in = 'gt_av', gx_in = 'gx_av',
                    r_out = 'radius',
                    gt_out = 'gt', gx_out = 'gx',
                    weight = 'norm_sum',
                   n_jack = n_jack,
                    ra = 'ra', dec = 'dec',                                       
                    z_name = 'cluster_z', obs_name = 'M200c',
                    Z_bin = Z_bin, Obs_bin = Mass_bin)

covariance_stack_jacknife_noshapenoise = prf.jacknife_covariance(profile = profile_noshapenoise,
                    r_in = 'radius_av',
                    gt_in = 'gt_av', gx_in = 'gx_av',
                    r_out = 'radius',
                    gt_out = 'gt', gx_out = 'gx',
                    weight = 'norm_sum',
                   n_jack = n_jack,
                    ra = 'ra', dec = 'dec',                                       
                    z_name = 'cluster_z', obs_name = 'M200c',
                    Z_bin = Z_bin, Obs_bin = Mass_bin)

print('ok jackknife')

In [ ]:
index_dm = 2

In [ ]:
profile_f['radial_bin'][0]

In [ ]:
fig, ax = plt.subplots(2,2, figsize = (10,10), sharex = True, sharey = True)
plt.rcParams['axes.linewidth']= 2
fig.subplots_adjust(wspace=0, hspace=0)
plt.tick_params(axis='both', which = 'major', labelsize= 15)
ax[0,0].set_title('Jackknife', fontsize = 20)
ax[0,1].set_title('Bootstrap', fontsize = 20)
ax[0,0].imshow(np.corrcoef(covariance_stack_jacknife_shapenoise['cov_t'][index_dm]), cmap ='bwr', vmin = -1, vmax = 1)
ax[0,1].imshow(np.corrcoef(covariance_stack_bootstrap_shapenoise['cov_t'][index_dm]), cmap ='bwr', vmin = -1, vmax = 1)
ax[1,0].imshow(np.corrcoef(covariance_stack_jacknife_noshapenoise['cov_t'][index_dm]), cmap ='bwr', vmin = -1, vmax = 1)
ax[1,1].imshow(np.corrcoef(covariance_stack_bootstrap_noshapenoise['cov_t'][index_dm]), cmap ='bwr', vmin = -1, vmax = 1)
index = np.arange(len(profile_f['radial_bin'][0]))
index = index[index[0]: index[-1]: 4]
ticks_ = profile_f['radial_bin'][0][index]
ticks_plot = [f'{t:.1f}' for t in ticks_]
for i in range(2):
    for j in range(2):
        ax[i,j].tick_params(axis='both', which = 'major', labelsize= 15)
        plt.sca(ax[i,j])
        plt.xticks(index, ticks_plot, color = 'k')
        plt.yticks(index, ticks_plot, color = 'k')
        if j == 0 : ax[i,j].set_ylabel(r'R [Mpc]', fontsize = 15)
        if i == 1 : ax[i,j].set_xlabel(r'R [Mpc]', fontsize = 15)

In [ ]:
S = 4*np.pi*15**2 #Mpc^2
n_gal_per_bin_average = []
ns_Mpc, ns_arcmin = [],[]
sigma_2_av = []
for p in profile_stack_noshapenoise:
    cluster_id = np.array(p['cluster_id'])
    mask = np.isin(profile_nsf['cluster_id'],cluster_id)
    dm_halos_cut = profile_nsf[mask]
    average_n_gal_per_bin = np.average(dm_halos_cut['n_gal'], axis = 0)
    average_sigma_2_per_bin = np.average(dm_halos_cut['sigma_crit^2_av'], axis = 0)
    n_gal_per_bin_average.append(average_n_gal_per_bin)
    average_n_gal_tot = np.sum(average_n_gal_per_bin)
    average_sigma_2 = np.mean(average_sigma_2_per_bin)
#    S = 4*np.pi*15**2 #Mpc^2
    ns_Mpc.append(average_n_gal_tot/S)
    theta_arcmin = 3437.75*(15/cosmo.eval_da(p['z_mean']))
    S_arcmin = np.pi*theta_arcmin**2
    ns_arcmin.append(average_n_gal_tot/S_arcmin)
    sigma_2_av.append(average_sigma_2_per_bin)
profile_stack_noshapenoise['ngal_per_bin'] = np.array(n_gal_per_bin_average)
profile_stack_noshapenoise['av_sigma^2'] = np.array(average_sigma_2)

In [ ]:
plt.rcParams['axes.linewidth']=2
index = index_dm
plt.figure(figsize = (10,8))
#label_z = f'{z_bin[0]:.2f} < z < {z_bin[1]:.2f}'
#label_M = f'{np.log10(m_bin[0]):.2f}' + r'$ < \log_{10}(M_{\rm 200c}) < $' + f'{np.log10(m_bin[1]):.2f}'
#plt.plot([], [], 'w', label = label_z)
#plt.plot([], [], 'w', label = label_M)
c = ['r','b']
fmt = ['-', '--']
marker = ['o', 's']
markersize = [2,2]
name = ['center cosmoDC2', 'center RedMapper']
profile_ = profile_stack_noshapenoise
#covariance = [covariance_RedMapper_stack_bootstrap, covariance_RedMapper_stack_jacknife, covariance_RedMapper_stack_sample]

f_cut = profile_[index_dm]
std_epsilon = 0.15
std_shear = 0.013

label_boot = f'Bootstrap'
label_jack = f'Jackknife'
label_sample = 'Sample'

gt_err_bootstrap = np.sqrt(f_cut['n_stack']*np.array(covariance_stack_bootstrap_shapenoise[index_dm]['cov_t'].T.diagonal()))
gt_err_jacknife  = np.sqrt(f_cut['n_stack']*np.array(covariance_stack_jacknife_shapenoise[index_dm]['cov_t'].T.diagonal()))#/covariance_RedMapper_stack_jacknife[index_cut]['Hartlap'][0]
gt_err_sample  = np.array(f_cut['n_stack']*covariance_stack_sample_shapenoise[index_dm]['cov_t'].T.diagonal())**(1/2)
gt_err_sample_true  = np.array(f_cut['n_stack']*covariance_stack_sample_noshapenoise[index_dm]['cov_t'].T.diagonal())**(1/2)

#print(gt_err_sample)
plt.plot(f_cut['radius'], gt_err_bootstrap,'cyan', label = label_boot + ' $-\ \sigma_\epsilon$', linewidth = 3)
plt.plot(f_cut['radius'], gt_err_sample_true,'--r', linewidth = 3, label = label_boot + ' $-\ \sigma_\gamma$')
plt.plot(f_cut['radius'], gt_err_jacknife,'.b', label = label_jack+ ' $-\ \sigma_\epsilon$', linewidth = 5)
plt.plot(f_cut['radius'], gt_err_sample,'xg', label = label_sample+ ' $-\ \sigma_\epsilon$', linewidth = 1, markersize = 10)

gt_err_th_epsilon = (av_sigma_2(f_cut['z_mean']) * std_epsilon**2/(f_cut['ngal_per_bin']))**(1/2)
gt_err_th_shear = (av_sigma_2(f_cut['z_mean']) *std_shear**2/(f_cut['ngal_per_bin']))**(1/2)
plt.plot(f_cut['radius'], gt_err_th_epsilon,'-k', zorder = 0, alpha = 1, linewidth = 3, label = r'shapenoise $-\ \sigma_\epsilon$')
plt.plot(f_cut['radius'], gt_err_th_shear,'-r', zorder = 0, alpha = 1, linewidth = 3, label = 'shapenoise $-\ \sigma_\gamma$')


plt.ylim(3e12, 9*1e14)
plt.xlim(min(f_cut['radius']), max(f_cut['radius']))
plt.xscale('log')
plt.yscale('log')
plt.legend(frameon = False, loc = 'upper right', fontsize = 18)
plt.xlabel('R [Mpc]', fontsize = 20)
plt.ylabel(r'$[N_{\rm cluster}Cov(\Delta\Sigma, \Delta\Sigma)]^{1/2}$ '+'$[$' + r'${\rm M}$' + r'$_\odot\;$'+ r'${\rm Mpc}$'+r'$^{-2}$'r'$]$', fontsize = 20)
plt.xticks(ticks=[0.1, 1,10], labels=['0.1', '1','10'])
plt.tick_params(axis='both', which = 'major', labelsize= 20)
#plt.savefig('/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/Images/covariance_diagonal.png', bbox_inches='tight', dpi=300)

In [ ]:
moo = clmm.Modeling(massdef = 'critical', delta_mdef = 200, halo_profile_model = 'nfw')
moo.set_cosmo(cosmo)
deff = ccl.halos.massdef.MassDef(200, 'critical', c_m_relation=None)
conc = ccl.halos.concentration.ConcentrationDiemer15(mdef=deff)

In [ ]:
def modele_ds(r, logm, c, cluster_z):
    m = 10.**logm 
    moo.set_mass(m), moo.set_concentration(c)
    deltasigma = []
    for i, xr in enumerate(r):
        deltasigma.append(moo.eval_excess_surface_density(xr, cluster_z))
    return np.array(deltasigma)

In [ ]:
from matplotlib.transforms import Affine2D
fig, ax = plt.subplots(figsize = (10,10))
z_bin = Z_bin[2]
m_bin = Mass_bin[2]
label_z = f'{z_bin[0]:.2f} < z < {z_bin[1]:.2f}'
label_M = f'{np.log10(m_bin[0]):.2f}' + r'$ < \log_{10}(M_{\rm 200c}) < $' + f'{np.log10(m_bin[1]):.2f}'
plt.plot([], [], 'w', label = label_z)
plt.plot([], [], 'w', label = label_M)
c = ['r','b']
fmt = ['-', '--']
marker = ['o', 's']
markersize = [2,2]
name = ['center cosmoDC2', 'center RedMapper']
profile_ = profile_stack_shapenoise
profile_true = profile_stack_noshapenoise
mask_z = (profile_['z_mean'] > z_bin[0])*(profile_['z_mean'] < z_bin[1])
mask_m = (profile_['obs_mean'] > m_bin[0])*(profile_['obs_mean'] < m_bin[1])
index = np.arange(len(profile_))
index_cut = index[mask_m * mask_z]
f_cut_true = profile_true[index_cut][0]
f_cut = profile_[index_cut][0]

gt_err_sample  = np.array(covariance_stack_bootstrap_shapenoise[index_cut]['cov_t'].T.diagonal()[0])**(1/2)
gt_err_sample_true  = np.array(covariance_stack_bootstrap_noshapenoise[index_cut]['cov_t'].T.diagonal()[0])**(1/2)

cst = 0.02

#ax.errorbar(f_cut_true['radius']- f_cut['radius']*cst, f_cut_true['gt'], gt_err_sample_true, fmt = '.', label = 'wo/ intrinsic ellipticity', linewidth = 2, capsize = 3, c = 'magenta')
#plt.fill_between(list(f_cut['radius']), list(f_cut['gt'] + gt_err_sample), list(f_cut['gt'] - gt_err_sample), color = 'cyan', alpha = 0.3, label = 'shapenoise')
#plt.fill_between(list(f_cut_true['radius']), list(f_cut_true['gt'] + gt_err_sample_true), list(f_cut_true['gt'] - gt_err_sample_true), color = 'magenta', alpha = 0.5, label = 'no shapenoise')
ax.errorbar(f_cut['radius'] + f_cut['radius']*cst, f_cut['gt'], gt_err_sample, fmt = '.', linewidth = 2, capsize = 4, c = 'b', markersize = 10, zorder = 1000)
#plt.errorbar(f_cut['radius'][0], f_cut['gt'][0],gt_err_sample,'.b', linewidth = 3)

cfiducial = conc._concentration(cosmo_ccl, f_cut['obs_mean'], 1/(1 + f_cut['z_mean']))
#cfiducial = Duffy_concentration(f_cut['obs_mean'], f_cut['z_mean'], 'critical')
fiducial = modele_ds(f_cut['radius'], np.log10(f_cut['obs_mean']), cfiducial, f_cut['z_mean'])
ax.plot(f_cut['radius'], fiducial,'--', c = 'g', alpha = 1, linewidth= 3, label = 'NFW 1-halo term')
#ax.grid(True)

n = f_cut['n_stack']
label_n = f'N = {n:.0f}'
#ax.plot([], [], 'w', label = label_n)
ax.set_ylim(9e11, 3*1e14)
ax.set_xlim(min(f_cut_true['radius']), max(f_cut_true['radius']))
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='both', which = 'major', labelsize= 18, zorder = 0)
ax.legend(frameon = False, loc = 'upper right', fontsize = 20)
ax.set_xlabel('R [Mpc]', fontsize = 22)
ax.set_ylabel(r'$\Delta\Sigma_+$ ' +'$[$' + r'${\rm M}$' + r'$_\odot\;$'+ r'${\rm Mpc}$'+r'$^{-2}$'r'$]$', fontsize = 22)
#plt.xticks(ticks=[0.1, 1,2, 3,6,10], labels=['0.1', '1','2', '3', '6','10'])
#plt.tick_params(axis='both', which = 'major', labelsize= 15)
#plt.savefig('/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/Images/t.png', bbox_inches='tight', dpi=300)

In [ ]:
fig, ax = plt.subplots(1,len(Z_bin), figsize = (30,5), sharey = True)
plt.rcParams['axes.linewidth']= 2
fig.subplots_adjust(wspace=0, hspace=0)
m_bin = Mass_bin[0]

for i,z in enumerate(Z_bin):
    mask_z = (profile_['z_mean'] > z[0])*(profile_['z_mean'] < z[1])
    f_cut = profile_[mask_z][0]
    gt_err_sample  = np.array(covariance_stack_bootstrap_shapenoise[index_cut]['cov_t'].T.diagonal()[0])**(1/2)
    ax[i].fill_between(list(f_cut['radius']), list(f_cut['gt'] + gt_err_sample), list(f_cut['gt'] - gt_err_sample), color = 'magenta', alpha = 0.3, label = 'shapenoise')
    
    cfiducial = conc._concentration(cosmo_ccl, f_cut['obs_mean'], 1/(1 + f_cut['z_mean']))
    #cfiducial = Duffy_concentration(f_cut['obs_mean'], f_cut['z_mean'], 'critical')
    fiducial = modele_ds(f_cut['radius'], np.log10(f_cut['obs_mean']), cfiducial, f_cut['z_mean'])
    ax[i].plot(f_cut['radius'], fiducial,'--', c = 'b', alpha = 1, linewidth= 3, label = 'fiducial')
    
    ax[i].set_ylim(5e11, 2*1e14)
    ax[i].set_xlim(min(f_cut_true['radius']), max(f_cut_true['radius']))
    #ax[i].vlines(1,5e11, 3*1e14, linestyle = '--', color = 'k')
    ax[i].set_xscale('log')
    ax[i].set_yscale('log')
    ax[i].tick_params(axis='both', which = 'major', labelsize= 18, zorder = 0)
    ax[i].set_xlabel('R [Mpc]', fontsize = 22)
    if i == 0:
        ax[i].set_ylabel(r'$\Delta\Sigma_+$ ' +'$[$' + r'${\rm M}$' + r'$_\odot\;$'+ r'${\rm Mpc}$'+r'$^{-2}$'r'$]$', fontsize = 22)
        ax[i].legend(frameon = False, loc = 'upper right', fontsize = 15)